In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import dask.dataframe as dd
from geopy.distance import geodesic

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#df = pd.read_csv('../csv/credit_card_fraud_2019_q1_15K_rows.csv')
#df = pd.read_csv('../csv/credit_card_fraud_2019_Dec.csv')
df = pd.read_csv('../csv/credit_card_fraud_sample_2019_Dec_100K.csv')

In [8]:
df['trans_num'].nunique()


2606359

In [ ]:
df_sample = df.sample(n=100000, random_state=42)
